# Photo Classifier: Model evaluation

## Imports

In [1]:
import random
import numpy as np
import pandas as pd
from collections import Counter
from ipywidgets import interact
import ipywidgets as widgets

In [2]:
from file_io import load_pickle_file
from constants import MODEL_FILE_PATH
from image_transforms import IMAGE_TRANSFORMS
from prediction import predict
from interpreter import Interpreter

In [3]:
import random
random.seed(42)

In [4]:
DATA_SET = 'test'

## Load model and data

In [5]:
model_data = load_pickle_file(MODEL_FILE_PATH)
model = model_data['model']
image_data = model_data['data']

## Prediction

In [6]:
x = image_data.get_images(DATA_SET)
y_true = image_data.get_classes(DATA_SET)

In [7]:
y_pred, probabilities = predict(x, model, IMAGE_TRANSFORMS[DATA_SET])

## Results interpretation

In [8]:
interpreter = Interpreter(x, y_pred, y_true, probabilities, image_data.class_to_label_mapping)

In [9]:
accuracy = interpreter.calculate_accuracy()
accuracy

0.76

In [10]:
confusion_matrix = interpreter.calculate_confusion_matrix()
confusion_matrix

Prediction,me,not_people,other_people
True,,,
me,7,0,0
not_people,0,11,0
other_people,1,5,1


In [11]:
accuracy_by_label = interpreter.calculate_accuracy_by_label()
accuracy_by_label

,is_correct
label,
me,1.000000
not_people,1.000000
other_people,0.142857


In [12]:
misclassified_samples = interpreter.get_misclassified_samples()
misclassified_samples

,true_label,predicted_label,propability_true,propability_predicted,is_correct
1,other_people,not_people,0.064768,0.934663,False
9,other_people,not_people,0.008145,0.990678,False
10,other_people,not_people,0.345194,0.641280,False
16,other_people,me,0.173591,0.813108,False
18,other_people,not_people,0.205489,0.587828,False
23,other_people,not_people,0.107438,0.766069,False


In [13]:
most_uncertain_samples = interpreter.get_most_uncertain_samples(5)
most_uncertain_samples

,probability
sample,
20,0.529680
4,0.537969
18,0.587828
10,0.641280
23,0.766069


In [14]:
most_incorrect_samples = interpreter.get_most_incorrect_samples(5)
most_incorrect_samples

,probability
sample,
9,0.008145
1,0.064768
23,0.107438
16,0.173591
18,0.205489


In [15]:
@interact(index=widgets.IntSlider(min=0, max=len(y_true)-1, step=1, value=0, continuous_update=False))
def show_prediction(index=0):
    interpreter.plot_prediction(index)

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=24), Output()), _do…